In this notebook, I'll show you how to save and restore models with Weights and Biases.

W&B lets you save everything you need to reproduce your models - weights, architecture, predictions, code to a safe place in the cloud.

This is useful because you don’t have to re-train your models, you can simply view their performance days, weeks, or even a few months later. Before you're ready to deploy, you can compare the performance of all the models you trained in the previous months and restore the best performing one.

# Train a model

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import wandb
from wandb.keras import WandbCallback

import wandb
from wandb.keras import WandbCallback

# define hyperparameters
defaults=dict(
    dropout = 0.2,
    hidden_layer_size = 128,
    layer_1_size = 16,
    layer_2_size = 32,
    learn_rate = 0.01,
    decay = 1e-6,
    momentum = 0.9,
    epochs = 5,
    )

wandb.login(key='f8fcf3355333e0c4f01595461dc68e14557e483c')
# initialize a new wandb run
wandb.init(project="github_actions_wandb_aws_ec2", config=defaults)
config = wandb.config

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

In [ ]:
# %wandb
# build model
model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# compile and train model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels)])

# Save a model

There are two ways to save a file to associate with a run.
1. Use wandb.save(filename).
2. Put a file in the wandb run directory, and it will get uploaded at the end of the run.

If you want to sync files as they're being written, you can specify a filename or glob in wandb.save.

See the docs for [frequently asked questions](https://docs.wandb.com/library/save#common-questions) about saving and restoring.

In [ ]:
# "model.h5" is saved in wandb.run.dir & will be uploaded at the end of training
model.save(os.path.join(wandb.run.dir, "model.h5"))

# Save a model file manually from the current directory:
wandb.save('model.h5')

# Save all files that currently exist containing the substring "ckpt":
wandb.save('../logs/*ckpt*')

# Save any files starting with "checkpoint" as they're written to:
wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))

# Restore a model

Restore a file, such as a model checkpoint, into your local run folder to access in your script.

Common use cases:
- restore the model architecture or weights generated by past runs
- resume training from the last checkpoint in the case of failure (see the section on resuming for crucial details)

See [the restore docs](https://docs.wandb.com/library/restore) for more details.

In [ ]:
# restore the model file "model.h5" from a specific run by user "lavanyashukla"
# in project "save_and_restore" from run "10pr4joa"
best_model = wandb.restore('model.h5', run_path="lavanyashukla/save_and_restore/10pr4joa")

# use the "name" attribute of the returned object
# if your framework expects a filename, e.g. as in Keras
model.load_weights(best_model.name)